In [ ]:
from extra_data import RunDirectory 
import h5py 
import numpy as np 
%matplotlib widget
import matplotlib.pyplot as plt 

In [ ]:
def extract_gifs(r, verbose=True):
    run = RunDirectory(path=f'/gpfs/exfel/exp/SPB/202202/p003046/raw/r0{r}')  
    
    # The require_all=True ensures we only get trains which exist for all sources
    sel = run.select([
        ('SPB_XTD9_XGM/XGM/DOOCS:output', 'data.intensitySa1TD'),
        ('SPB_IRU_MOTORS/MDL/DATA_SELECT', '*'),
        ('SPB_EXP_ZYLA/CAM/1:daqOutput', 'data.image.pixels'),
    ], require_all=True)
    if(verbose):
        sel.info()   
    sidemic_extra = sel['SPB_EXP_ZYLA/CAM/1:daqOutput', 'data.image.pixels']
    trains = sidemic_extra.train_id_coordinates()
    sidemic = sidemic_extra.xarray()
    return sidemic

In [ ]:
import imageio
def write_gifs(run):
    sidemic = extract_gifs(run)
    # convert to 8bit and select only the area around the liquid sheet
    sidemic_8bit = np.array(np.minimum(255,np.maximum(0,(255*(sidemic.data[:,200:1400,200:600]-np.percentile(sidemic[0], 1))/(np.percentile(sidemic[0], 99)-np.percentile(sidemic[0], 1))))).astype('uint8'))
    step = 5
    with imageio.get_writer('sidemic_%d.gif' % (run), mode='I', duration=0.1) as writer:
        for i in np.arange(stop=sidemic.shape[0],step=step):
            writer.append_data(sidemic_8bit[i])
        writer.close()

In [ ]:
write_gifs(195)

In [ ]:
for i in np.arange(184,198):
    write_gifs(i)